Work In Progress

License: Attribution-ShareAlike 4.0 International
Dataset: https://www.kaggle.com/datasets/wikimedia-foundation/wikipedia-structured-contents/data

In [ ]:
# Install packages

!pip install kagglehub
!pip install tqdm
!pip install scikit-learn

In [ ]:
import kagglehub
import json
import pandas as pd
from tqdm import tqdm

# Downloads the Wikipedia structured contents dataset from Kaggle.

file_path = kagglehub.dataset_download("wikimedia-foundation/wikipedia-structured-contents",path="enwiki_namespace_0/enwiki_namespace_0_0.jsonl")
df = pd.read_json(file_path, lines=True)
print(f"Successfully loaded {len(df)} records")

# Saves it to the given path.
df.to_parquet('my-data-0_0.parquet', engine='pyarrow', compression='snappy')

# Prints how many records (rows) were loaded.
print("Parquet file saved successfully!")

# Saves the DataFrame to a .parquet file (a compressed, fast format for large datasets).
df_read = pd.read_parquet('my-data-0_0.parquet')

# Confirms that the file was saved.
print(df.shape)

In [ ]:
# import libraries
import pandas as pd

# Reads the Parquet file.
read_parquet_df = pd.read_parquet("my-data-0_0.parquet")

# Converts the DataFrame to a JSON string.
read_json_str_00 = read_parquet_df.to_json()

# Saves that JSON string into a .csv file (even though it's JSON format — kind of a misnaming).
with open("data-0_0.csv","w") as file:
    file.write( read_json_str_00 + "\n")

In [ ]:
# # import libraries
import pandas as pd

# # Reads the Parquet file.
read_parquet_df = pd.read_parquet("my-data-0_0.parquet")

# # Converts the DataFrame to a JSON string.
# read_json_str_00 = read_parquet_df.to_json()
with open("data-0_0.json", "w") as f:
    f.write(read_json_str_00)

# # Saves that JSON string into a .csv file (even though it's JSON format — kind of a misnaming).
with open("data-0_0.csv","w") as file:
    file.write( read_json_str_00 + "\n")

In [ ]:
# import libraries
import pandas as pd
import json
import csv
from sklearn.model_selection import train_test_split

# Load the JSON file into a Python dictionary
with open('data-0_0.json', 'r') as f:
    data_dict = json.load(f)

# Flatten the dictionary into a list
data_list = []
for key, value in data_dict.items():
    if isinstance(value, list):
        for item in value:
            data_list.append({'Source': key, 'Target': item})
    else:
        data_list.append({'Source': key, 'Target': value})

# Create a DataFrame
df = pd.DataFrame(data_list)

# Splitting the dataframe into train data
train_df , _ = train_test_split(df, test_size=0.0001, random_state=42)
print(train_df.head(10))




# Select specific rows where 'Source' is 'sections'
selected_rows = train_df[train_df['Source'] == 'sections']

# test output
# print(selected_rows.head(10))

text_list = []

# Extract text from nested structure
if not selected_rows.empty:
    selected_target_rows = selected_rows['Target']

    for entry in selected_target_rows:
        if isinstance(entry, dict):  # <-- Always good to check
            for key, sections in entry.items():
                if isinstance(sections, list):  # Check if sections is a list
                    for section in sections:
                        has_parts = section.get('has_parts')
                        if has_parts:
                            for part in has_parts:
                                value = part.get('value')
                                if value is not None:
                                    text_list.append(value)

# Print first 2 results
if text_list:
    print(text_list[0])
    print(text_list[1])
    

    with open('out.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(text_list)
else:
    print("No text extracted.")

In [ ]:
# import nltk
# import re
# import spacy
# import pandas as pd
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures

# # Download required resources
# nltk.download('punkt')
# nltk.download('stopwords')
# spacy.cli.download("en_core_web_lg")
# nlp = spacy.load("en_core_web_lg")

# STOPWORDS = set(stopwords.words('english'))

# def clean_text(text):
#     text = str(text).lower().strip()
#     text = re.sub(r'\s+', ' ', text)
#     text = re.sub(r'#[\w]+|@[\w]+', lambda match: match.group(0), text)
#     text = re.sub(r'[^a-zA-Z0-9\s#@]', '', text)
#     text = re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", text)
#     return text

# # Read CSV
# df = pd.read_csv('out.csv', on_bad_lines='skip', dtype=str)
# # Combine all character columns into one string per row
# df['Extracted_Text'] = df.fillna('').astype(str).agg(''.join, axis=1)

# # Sample 10,000 rows
# df_sample = df.sample(n=10000, random_state=42)

# # Clean text
# df_sample['Cleaned_Extracted_Text'] = df_sample['Extracted_Text'].apply(clean_text)

# def tokenize_and_remove_stopwords(text):
#     tokens = word_tokenize(text)
#     return [word for word in tokens if word not in STOPWORDS]

# # Tokenize and remove stopwords
# df_sample['Tokens'] = df_sample['Cleaned_Extracted_Text'].apply(tokenize_and_remove_stopwords)

# # Combine all tokens
# all_words = []
# for tokens in df_sample['Tokens']:
#     all_words.extend(tokens)

# # Find bigrams
# bigram_measures = BigramAssocMeasures()
# finder = BigramCollocationFinder.from_words(all_words)
# finder.apply_freq_filter(2)

# top_bigrams = finder.nbest(bigram_measures.raw_freq, 10)
# top_bigrams_500 = finder.nbest(bigram_measures.raw_freq, 500)


# # Output
# print(df_sample[['Cleaned_Extracted_Text','Tokens']].head(10))
# print("Rows after sampling:", len(df_sample))
# print("Top 10 bigrams by raw freq:", top_bigrams)
# print("Top 10 bigrams by raw freq:", top_bigrams_500)

In [ ]:
import nltk
import re
import spacy
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from collections import Counter

# Download required resources
nltk.download('punkt')
nltk.download('stopwords')
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower().strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'#[\w]+|@[\w]+', lambda match: match.group(0), text)
    text = re.sub(r'[^a-zA-Z0-9\s#@]', '', text)
    text = re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", text)
    return text

# Read CSV
df = pd.read_csv('out.csv', on_bad_lines='skip', dtype=str)
# Combine all character columns into one string per row
df['Extracted_Text'] = df.fillna('').astype(str).agg(''.join, axis=1)

# Sample 10,000 rows
df_sample = df.sample(n=10000, random_state=42)

# Clean text
df_sample['Cleaned_Extracted_Text'] = df_sample['Extracted_Text'].apply(clean_text)

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    return [word for word in tokens if word not in STOPWORDS]

# Tokenize and remove stopwords
df_sample['Tokens'] = df_sample['Cleaned_Extracted_Text'].apply(tokenize_and_remove_stopwords)

# Combine all tokens
all_words = []
for tokens in df_sample['Tokens']:
    all_words.extend(tokens)

# Find bigrams
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(all_words)
finder.apply_freq_filter(2)

top_bigrams = finder.nbest(bigram_measures.raw_freq, 10)
top_bigrams_500 = finder.nbest(bigram_measures.raw_freq, 500)

# Count frequencies
word_freq = Counter(all_words)

# Get most common words
most_common_words = word_freq.most_common(100)  

# Create a set of common words
common_words_set = set(word for word, _ in most_common_words)

# Output
print(df_sample[['Cleaned_Extracted_Text','Tokens']].head(10))
print("Rows after sampling:", len(df_sample))
print("Top 10 bigrams by raw freq:", top_bigrams)
print("Top 10 bigrams by raw freq:", top_bigrams_500)
print(most_common_words)

most_common_words_in_bigrams = []
for bigram in top_bigrams_500:
    most_common_words_in_bigrams.append((bigram[0], bigram[1]))  # Fix: Append as a tuple

filtered_bigrams_partially_common = [
    bigram for bigram in top_bigrams_500 
    if bigram[0] in common_words_set or bigram[1] in common_words_set
]

print("Filtered bigrams (at least one word common):", filtered_bigrams_partially_common[:20])

all_pmi_bigrams = finder.score_ngrams(bigram_measures.pmi)

# Print them
print("All bigrams scored by PMI:")
for bigram, score in all_pmi_bigrams:
    print(bigram, score)

# Likelihood Ratio:
all_lr_bigrams = finder.score_ngrams(bigram_measures.likelihood_ratio)

print("\nAll bigrams scored by Likelihood Ratio:")
for bigram, score in all_lr_bigrams:
    print(bigram, score)
